In [76]:
!killall streamlit
!killall ngrok
!pip install streamlit pyngrok pandas numpy matplotlib > /dev/null 2>&1

In [77]:
%%writefile mindtrack_app.py
import streamlit as st
import datetime
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# ----------------------------
# Custom "Blue Theme" + Snowfall CSS
# ----------------------------
st.markdown(
    """
    <style>
    /* Main app background */
    .stApp {
        background-color: #0a192f; /* Dark Navy Blue */
        color: #ffffff; /* White text */
        overflow: hidden; /* Hide scrollbars from snow */
    }
    /* This makes the top header bar match the app background */
    [data-testid="stHeader"] {
        background: transparent;
    }

    /* Headings and Titles */
    h1, h2, h3, h4, h5, h6, .stTitle, .stHeader {
        color: #ccd6f6; /* Light Blue-ish White */
    }

    /* Buttons */
    .stButton > button {
        background-color: #34ebc6; /* Bright Teal/Mint */
        color: #0a192f; /* Dark text for contrast */
        border: none;
        border-radius: 8px;
        font-weight: bold;
    }
    .stButton > button:hover {
        background-color: #2bc1a4; /* Darker teal on hover */
        color: #0a192f;
    }

    /* Radio buttons */
    .stRadio > label {
        background-color: #172a45; /* Lighter navy for widget bg */
        padding: 10px;
        border-radius: 8px;
        margin-bottom: 5px;
    }

    /* Checkboxes */
    .stCheckbox {
        accent-color: #34ebc6; /* Teal check */
    }

    /* Progress bars */
    .stProgress > div > div {
        background-color: #34ebc6; /* Teal progress */
    }

    /* Text Inputs and Selectboxes */
    .stTextInput > div > div > input,
    .stSelectbox > div > div {
        background-color: #172a45; /* Lighter navy bg */
        color: #ffffff;
        border-radius: 8px;
        border: 1px solid #34ebc6; /* Teal border */
    }

    /* Metric boxes */
    .stMetric {
        background-color: #172a45;
        padding: 15px;
        border-radius: 10px;
    }

    /* Success/Info/Warning messages */
    .stSuccess {
        background-color: #172a45;
        border: 1px solid #34ebc6;
        border-radius: 8px;
    }
    .stInfo {
        background-color: #172a45;
        border: 1px solid #1e90ff; /* Blue for info */
        border-radius: 8px;
    }
    .stWarning {
        background-color: #172a45;
        border: 1px solid #ffc107; /* Yellow for warning */
        border-radius: 8px;
    }

    /* --- Snowfall Effect --- */
    .snow {
        position: fixed;
        top: -10vh; /* Start above the screen */
        left: 0;
        width: 100%;
        height: 100vh;
        z-index: 0; /* Place behind other content */
        pointer-events: none; /* Let clicks pass through */
    }

    .snow .snowflake {
        position: absolute;
        width: 10px;
        height: 10px;
        background: white;
        border-radius: 50%;
        opacity: 0.6;
        animation: fall linear infinite;
    }

    /* Different sizes, speeds, and starting positions */
    .snowflake:nth-of-type(1) { width: 8px; height: 8px; animation-duration: 12s; animation-delay: -1s; left: 10%; }
    .snowflake:nth-of-type(2) { width: 12px; height: 12px; animation-duration: 15s; animation-delay: -3s; left: 20%; }
    .snowflake:nth-of-type(3) { width: 6px; height: 6px; animation-duration: 10s; animation-delay: -2s; left: 30%; }
    .snowflake:nth-of-type(4) { width: 10px; height: 10px; animation-duration: 13s; animation-delay: -5s; left: 40%; }
    .snowflake:nth-of-type(5) { width: 8px; height: 8px; animation-duration: 11s; animation-delay: -4s; left: 50%; }
    .snowflake:nth-of-type(6) { width: 12px; height: 12px; animation-duration: 16s; animation-delay: -6s; left: 60%; }
    .snowflake:nth-of-type(7) { width: 7px; height: 7px; animation-duration: 10s; animation-delay: -1s; left: 70%; }
    .snowflake:nth-of-type(8) { width: 10px; height: 10px; animation-duration: 14s; animation-delay: -3s; left: 80%; }
    .snowflake:nth-of-type(9) { width: 6px; height: 6px; animation-duration: 12s; animation-delay: -7s; left: 90%; }
    .snowflake:nth-of-type(10) { width: 9px; height: 9px; animation-duration: 18s; animation-delay: -8s; left: 95%; }
    .snowflake:nth-of-type(11) { width: 8px; height: 8px; animation-duration: 13s; animation-delay: -3s; left: 15%; }
    .snowflake:nth-of-type(12) { width: 11px; height: 11px; animation-duration: 17s; animation-delay: -5s; left: 25%; }
    .snowflake:nth-of-type(13) { width: 7px; height: 7px; animation-duration: 11s; animation-delay: -6s; left: 35%; }
    .snowflake:nth-of-type(14) { width: 10px; height: 10px; animation-duration: 15s; animation-delay: -9s; left: 45%; }
    .snowflake:nth-of-type(15) { width: 8px; height: 8px; animation-duration: 12s; animation-delay: -10s; left: 55%; }

    @keyframes fall {
        from {
            transform: translateY(0) translateX(0);
        }
        to {
            transform: translateY(110vh) translateX(20px); /* Fall down and slightly to the side */
        }
    }
    /* --- End Snowfall --- */

    </style>

    <div class="snow">
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
        <div class="snowflake"></div>
    </div>
    """,
    unsafe_allow_html=True
)
# ----------------------------
# End Custom CSS
# ----------------------------


# ----------------------------
# App Setup
# ----------------------------
st.set_page_config(page_title="MindTrack – Personal Wellness Tracker", layout="centered")
st.title("🧠 MindTrack – Personal Wellness, Mood & Goal Tracker")
st.caption("Your AI-powered wellness companion — build habits, track moods, and hit your goals!")


today = datetime.date.today()


# ----------------------------
# Initialize Data
# ----------------------------
if 'habits' not in st.session_state:
    st.session_state.habits = []
if 'checkins' not in st.session_state:
    st.session_state.checkins = []
if 'moods' not in st.session_state:
    st.session_state.moods = []
if 'goals' not in st.session_state:
    st.session_state.goals = {}
if 'motivation' not in st.session_state:
    st.session_state.motivation = random.choice([
        "Progress over perfection!",
        "You're doing amazing — keep it up!",
        "Habits make success inevitable.",
        "Consistency beats intensity!"
    ])


# ----------------------------
# Habit Management
# ----------------------------
st.header("📋 Habit Tracker")


col1, col2 = st.columns([3, 1])
with col1:
    new_habit = st.text_input("Add a new habit (e.g., Meditate, Walk, Journal):")
with col2:
    if st.button("➕ Add Habit"):
        if new_habit.strip():
            st.session_state.habits.append(new_habit.strip())
            st.success(f"Added habit: {new_habit.strip()}")
        else:
            st.warning("Please enter a valid habit name!")


if st.session_state.habits:
    delete = st.selectbox("🗑 Remove a habit", ["None"] + st.session_state.habits)
    if delete != "None" and st.button("Confirm Delete"):
        st.session_state.habits.remove(delete)
        st.session_state.checkins = [c for c in st.session_state.checkins if c[0] != delete]
        st.success(f"Deleted habit: {delete}")


# ----------------------------
# Daily Habit Check-ins
# ----------------------------
st.header("✅ Daily Check-ins")
st.write(f"Today is: **{today.strftime('%A, %d %B %Y')}**")


for habit in st.session_state.habits:
    checked = st.checkbox(f"{habit}", key=f"{habit}_{today}")
    if checked:
        if (habit, str(today)) not in st.session_state.checkins:
            st.session_state.checkins.append((habit, str(today)))
    else:
        st.session_state.checkins = [c for c in st.session_state.checkins if not (c[0] == habit and c[1] == str(today))]


# ----------------------------
# Mood Tracker
# ----------------------------
st.header("🙂 Mood Tracker")


mood_options = {"😊 Happy": 3, "😐 Okay": 2, "😔 Sad": 1}
mood_choice = st.radio("How do you feel today?", list(mood_options.keys()), horizontal=True)


if st.button("💾 Save Mood"):
    st.session_state.moods.append((str(today), mood_choice))
    st.success(f"Mood '{mood_choice}' saved for today!")


if st.session_state.moods:
    mood_df = pd.DataFrame(st.session_state.moods, columns=["Date", "Mood"])
    mood_df["Date"] = pd.to_datetime(mood_df["Date"])
    mood_df["Score"] = mood_df["Mood"].map(mood_options)

    st.subheader("📈 Mood Trend")
    # --- Style the plot to match the theme ---
    fig, ax = plt.subplots(figsize=(6,3))
    fig.patch.set_facecolor('white') # Match card background
    ax.set_facecolor('white') # Match card background

    ax.plot(mood_df["Date"], mood_df["Score"], marker="o", linewidth=2, color="#0a192f") # Teal line

    ax.set_yticks([1,2,3])
    ax.set_yticklabels(["😔", "😐", "😊"])

    # Set text and spine colors to white
    ax.set_xlabel("Date", color="black")
    ax.set_title("Mood Over Time", color="black")
    ax.tick_params(colors="black", which="both")
    for spine in ax.spines.values():
        spine.set_edgecolor("black") # Borders of the plot area in black

    # --- ADD THESE TWO LINES FOR BETTER DATE VISIBILITY ---
    plt.xticks(rotation=45, ha='right') # Rotate labels for better visibility
    plt.tight_layout() # Adjust plot to prevent labels from being cut off
    # --- END ADDITION ---

    # --- End plot styling ---
    st.pyplot(fig)


# ----------------------------
# Analytics
# ----------------------------
st.header("📊 Analytics & Streaks")


if st.session_state.habits and st.session_state.checkins:
    df = pd.DataFrame(st.session_state.checkins, columns=["Habit", "Date"])
    df["Date"] = pd.to_datetime(df["Date"])

    st.metric("Total Habits", len(st.session_state.habits))
    st.metric("Total Check-ins", len(df))

    st.subheader("7-Day Habit Progress")
    for habit in st.session_state.habits:
        recent = df[(df["Habit"] == habit) & (df["Date"] >= pd.to_datetime(today - datetime.timedelta(days=6)))]
        pct = round(len(recent) / 7 * 100, 1)
        st.progress(pct / 100)
        st.write(f"{habit}: **{pct}%** completed this week")

    # ----------------------------
    # Streak Calendar
    # ----------------------------
    st.subheader("📅 7-Day Streak Calendar")
    start_date = today - datetime.timedelta(days=6)
    days = [(start_date + datetime.timedelta(days=i)) for i in range(7)]

    for habit in st.session_state.habits:
        st.markdown(f"**🔥 {habit}**")
        row = []
        for d in days:
            # --- THIS IS THE FIXED LINE ---
            done = ((df["Habit"] == habit) & (df["Date"].dt.date == d)).any()
            # --- END OF FIX ---

            color = "#34eb43" if done else "#e0e0e0"
            emoji = "✅" if done else "⬜"
            cell = f"<div style='display:inline-block;width:55px;height:55px;background-color:{color};border-radius:10px;text-align:center;line-height:55px;font-size:22px;margin:2px'>{emoji}</div>"
            row.append(cell)
        st.markdown("".join(row), unsafe_allow_html=True)
else:
    st.info("Start tracking your habits to view progress and streaks!")


# ----------------------------
# 🎯 Weekly Goals Dashboard
# ----------------------------
st.header("🎯 Weekly Goal Dashboard")


goal_habit = st.selectbox("Select a habit to set a goal for:", ["None"] + st.session_state.habits)
if goal_habit != "None":
    goal_target = st.number_input("Set your weekly goal (e.g., 5 days):", min_value=1, max_value=7, value=5)
    if st.button("✅ Save Goal"):
        st.session_state.goals[goal_habit] = goal_target
        st.success(f"Goal set: Complete '{goal_habit}' {goal_target} times this week!")


if st.session_state.goals:
    st.subheader("Your Weekly Goal Progress")
    for habit, target in st.session_state.goals.items():
        df = pd.DataFrame(st.session_state.checkins, columns=["Habit", "Date"])
        df["Date"] = pd.to_datetime(df["Date"])
        this_week = df[(df["Habit"] == habit) & (df["Date"] >= pd.to_datetime(today - datetime.timedelta(days=6)))]
        completed = len(this_week)
        progress = min(completed / target * 100, 100)
        st.write(f"**{habit}** – {completed}/{target} days")
        st.progress(progress / 100)
        if progress >= 100:
            st.success("🎉 Goal achieved!")
        elif progress >= 60:
            st.info("💪 You're on track!")
        else:
            st.warning("⚡ Try to be more consistent!")


# ----------------------------
# 🤖 AI Smart Suggestions
# ----------------------------
st.header("🤖 Smart Habit Suggestions")


suggestions = [
    "Meditate for 10 minutes",
    "Read 5 pages of a book",
    "Drink 2L of water",
    "Go for a walk",
    "Stretch before bed",
    "Write gratitude journal",
    "Sleep before 11 PM"
]


if st.session_state.moods:
    avg_mood = np.mean([mood_options[m] for _, m in st.session_state.moods])
    if avg_mood < 1.5:
        st.warning("You seem a bit low lately 😔 — try something relaxing:")
        st.info(random.choice(["Meditate for 10 minutes", "Go for a walk", "Listen to music"]))
    elif avg_mood < 2.5:
        st.info("You’re doing okay 😐 — boost your energy with:")
        st.info(random.choice(["Read 5 pages", "Stretch or yoga"]))
    else:
        st.success("You're feeling great! 😊 Keep growing:")
        st.info(random.choice(["Learn something new", "Try a 15-min workout"]))
else:
    st.info("Once you log a few moods, MindTrack will suggest personalized habits!")


# ----------------------------
# 💬 Motivation
# ----------------------------
st.header("💬 Daily Motivation")
st.success(st.session_state.motivation)
if st.button("🔄 Refresh Quote"):
    st.session_state.motivation = random.choice([
        "A little progress each day adds up to big results.",
        "You are your best project — keep going!",
        "Consistency is key — don’t break the chain.",
        "Celebrate every small win!"
    ])
    st.rerun()


# ----------------------------
# 📤 Data Export
# ----------------------------
st.header("📤 Export Your Data")
if st.session_state.checkins or st.session_state.moods:
    habit_df = pd.DataFrame(st.session_state.checkins, columns=["Habit", "Date"])
    mood_df = pd.DataFrame(st.session_state.moods, columns=["Date", "Mood"])
    habit_csv = habit_df.to_csv(index=False).encode('utf-8')
    mood_csv = mood_df.to_csv(index=False).encode('utf-8')

    st.download_button("⬇️ Download Habits Data (CSV)", data=habit_csv, file_name="mindtrack_habits.csv", mime="text/csv")
    st.download_button("⬇️ Download Mood Data (CSV)", data=mood_csv, file_name="mindtrack_moods.csv", mime="text/csv")
else:
    st.write("No data available yet. Start tracking!")

Overwriting mindtrack_app.py


In [78]:
from pyngrok import ngrok, conf
import os
import subprocess
import time

# --- IMPORTANT ---
# Paste your NGROK_AUTHTOKEN here
NGROK_AUTHTOKEN = "Your ngrok authtoken here"

# --- Authenticate ngrok ---
conf.get_default().auth_token = NGROK_AUTHTOKEN
os.system(f"ngrok authtoken {NGROK_AUTHTOKEN}")

# --- Start the Streamlit app in the background ---
print("Starting Streamlit app... ⏳")
# We use subprocess.Popen for a more reliable background process
proc = subprocess.Popen(
    ["streamlit", "run", "mindtrack_app.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# --- Wait 15 seconds for the app to fully start ---
print("Waiting 15 seconds for the app to initialize...")
time.sleep(15)

# --- Connect to ngrok ---
try:
    print("Connecting to ngrok... 🚀")
    public_url = ngrok.connect(8501, "http")
    print("\n" + "="*50)
    print("✅ Your Streamlit app is live at:")
    print(public_url)
    print("="*50 + "\n")

except Exception as e:
    print(f"\n❌ Could not connect to ngrok. Error: {e}")

Starting Streamlit app... ⏳
Waiting 15 seconds for the app to initialize...
Connecting to ngrok... 🚀

✅ Your Streamlit app is live at:
NgrokTunnel: "https://35f72cba2370.ngrok-free.app" -> "http://localhost:8501"

